<a href="https://colab.research.google.com/github/ethanmjansen/BS4_Scraper/blob/master/Copy_of_THE_GOAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports for LSTM
!pip install tensorflow==2.0.0
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from keras.datasets import imdb 

In [ ]:
#download the data
top_words = 5000 
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [ ]:
X_train.shape

(25000,)

In [ ]:
#reverse lookup
import tensorflow
word_to_id = tensorflow.keras.datasets.imdb.get_word_index()
word_to_id = {k:(v+3) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in X_train[0] ))

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD

In [ ]:
max_review_length = 500 

In [ ]:
# Padding the sequences with zeros
print('Pad Sequences')
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print('X_train shape ', X_train.shape)
print('X_test shape ', X_test.shape)

Pad Sequences
X_train shape  (25000, 500)
X_test shape  (25000, 500)


In [ ]:
# Set embedding vector length
embedding_vector_length = 32 
# Instantiate model
model = Sequential(name='JanLSTM') 

# Model architecture 
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length, name='embedding_layer')) 
model.add(LSTM(100, name='LSTM_layer')) 
model.add(Dense(1, activation='sigmoid', name='dense_layer')) 

# Compile model
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 

# Model summary
model.summary() 

Model: "JanLSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 500, 32)           160000    
_________________________________________________________________
LSTM_layer (LSTM)            (None, 100)               53200     
_________________________________________________________________
dense_layer (Dense)          (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Train the model
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=3, batch_size=64) 

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 493s 20ms/sample - loss: 0.4170 - accuracy: 0.8033 - val_loss: 0.4022 - val_accuracy: 0.8300
Epoch 2/3
25000/25000 [==============================] - 483s 19ms/sample - loss: 0.2946 - accuracy: 0.8822 - val_loss: 0.5911 - val_accuracy: 0.8125
Epoch 3/3
25000/25000 [==============================] - 482s 19ms/sample - loss: 0.3145 - accuracy: 0.8677 - val_loss: 0.3209 - val_accuracy: 0.8666


In [ ]:
#Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.66%


In [ ]:
from numpy import array

In [ ]:
#predict sentiment from reviews
bad = "i hated this movie so much it sucked nuts"
good = "this movie was the best one i have ever seen"
for review in [good,bad]:
    tmp = []
    for word in review.split(" "):
        tmp.append(word_to_id[word])
    tmp_padded = sequence.pad_sequences([tmp], maxlen=max_review_length) 
    print("%s. Sentiment: %s" % (review,model.predict(array([tmp_padded][0]))[0][0]))


this movie was the best one i have ever seen. Sentiment: 0.89152753
i hated this movie so much it sucked nuts. Sentiment: 0.16989759
